In [ ]:
if True:
    from julia.api import Julia
    jl = Julia(compiled_modules=False)

#import julia; julia.install(quiet=True)
from julia import Main

import numpy     as np
import panel     as pn; pn.extension()
import holoviews as hv; hv.extension( "bokeh", logo=False)

from panel.interact import interact

def raster(img):  return hv.Image(img).opts(cmap="gray", xaxis=None, yaxis=None, frame_width=200, aspect='equal')

In [ ]:
%load_ext julia.magic

In [3]:
%%julia
using LinearAlgebra;

<div style="height:2cm;">
<div style="float:center;width:100%;text-align:center;"><strong style="height:100px;color:darkred;font-size:40px;">The Grassmannian</strong>
</div></div>

The set of all $k$-dimensional subspaces of $\mathbb{R}^n$ forms a smooth manifold: the Grassmannian $\mathrm{Gr}(k, n)$.

We introduce this space, its geometry, and how subspace distances naturally live on it.


### Optional: Wedge Products and Subspaces

The wedge product $v_1 \wedge \dots \wedge v_k$ represents a $k$-dimensional volume  
spanned by the vectors $v_1, \dots, v_k$.

In later algebra, this multivector can also encode the subspace itself  
(up to scale and orientation).

See: [Wedge Product and Volume](13_WedgeProduct.ipynb)  
for a visual and geometric introduction to wedge products.


## The Grassmannian

The set of all $k$-dimensional subspaces of $\mathbb{R}^n$  
forms a smooth manifold: the **Grassmannian** $\mathrm{Gr}(k, n)$.

It provides a geometric space where subspaces become points,  
and distances between them become paths.

---

### Definition

Let $k \leq n$.  
The Grassmannian $\mathrm{Gr}(k, n)$ is the set of all $k$-dimensional linear subspaces of $\mathbb{R}^n$.

Each point on $\mathrm{Gr}(k, n)$ is a subspace,  
not a basis or a set of vectors.

---

### Representations

A subspace $U \subset \mathbb{R}^n$ can be represented by:

- An orthonormal basis matrix $U \in \mathbb{R}^{n \times k}$  
- The projection operator $P = UU^\top$  
- A point on $\mathrm{Gr}(k, n)$

Different bases → same projector → same point on the Grassmannian.

---

### Geometry

The Grassmannian has a natural distance function  
defined by the principal angles between subspaces.

- Zero distance ⇔ subspaces are identical  
- Maximal distance ⇔ subspaces are orthogonal

Subspace distances become **geodesic distances** on $\mathrm{Gr}(k, n)$.

---

### Visualization

- $\mathrm{Gr}(1, 2)$ is a circle  
- $\mathrm{Gr}(1, 3)$ is the real projective plane $\mathbb{RP}^2$  
- Higher $\mathrm{Gr}(k, n)$ are curved manifolds

We can visualize motion on the Grassmannian as  
smooth paths between subspaces.

---

### Next

We now explore **geodesics** and optimization paths on $\mathrm{Gr}(k, n)$.
